In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from prophet import Prophet
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("item_orders.csv")

In [3]:
def get_time_series(df_in, filter_in, frequency = "Daily"):
    
    """
    Returns the time series dataframe for given filter, frequency,  and imput dataframe
    
    df_in = Pandas dataframe
    filter_in = filter that will be applied on df_in dataframe. Example: df_in.group1 == "Running shoes"
    frequency = Daily, Weekly or Monthly
    """
    
    df_temp = df_in[filter_in]
    if frequency == "Daily":
        df_temp['date']=pd.to_datetime(df_temp['date'], format = "%Y-%m-%d")
        df_out = df_temp.groupby("date").sum()
    
    if frequency == "Weekly":
        date=pd.to_datetime('2019-06-01')
        df_temp['date']=pd.to_datetime(df_temp['date'], format = "%Y-%m-%d")
        df_out = df_temp.resample('W-{:%a}'.format(date), on='date').sum()
        df_out = df_out.iloc[1: , :] # dropping the first row as it is an incomplete week
    
    if frequency == "Monthly":
        date=pd.to_datetime('2019-06-01')
        df_temp['date']=pd.to_datetime(df_temp['date'], format = "%Y-%m-%d")
        df_out = df_temp.resample('M'.format(date), on='date').sum()
    
    return(df_out["quantity"])
    

In [4]:
df.head()

,order_id,date,item_code,unit_price_vat_excl,quantity,department,item_name,name,group1,country
0,2000093387,2020-04-24 00:00:00,S101,3.506048,1,E-COMMER,Dopravné,- žádný výrobce -,NaN,Hungary
1,2000093391,2020-04-24 00:00:00,S101,3.737403,1,E-COMMER,Dopravné,- žádný výrobce -,NaN,Slovakia
2,2000093394,2020-04-24 00:00:00,S101,3.171318,1,E-COMMER,Dopravné,- žádný výrobce -,NaN,Czech Republic
3,2000093395,2020-04-24 00:00:00,S101,2.850775,1,E-COMMER,Dopravné,- žádný výrobce -,NaN,Czech Republic
4,2000093400,2020-04-24 00:00:00,S101,3.336224,1,E-COMMER,Dopravné,- žádný výrobce -,NaN,Romania


In [5]:
sales_counts = df["group1"].value_counts()
interest_items = list(sales_counts.index[sales_counts.index != "nan"][0:10])

In [6]:
interest_items.append("All items")

In [7]:
sales_counts[sales_counts.index.isin(interest_items)].sum()/df.shape[0]

0.6585047821726183

65% of sales fall into one of these categories

In [8]:
interest_items

['Running shoes',
 'Pants',
 'T-Shirts',
 'Football shoes',
 'Socks',
 'Other Footwear',
 'Sweatshirts',
 'Jackets',
 'Jerseys',
 'Fitness Shoes',
 'All items']

In [9]:
countries = list(df["country"].value_counts().index)
countries.append("All countries")
countries.remove("Other") # some items under Other countries have so little observations that they are causing errors in the model
countries

['Czech Republic',
 'Slovakia',
 'Romania',
 'Hungary',
 'Germany',
 'Spain',
 'France',
 'Italy',
 'Croatia',
 'Austria',
 'All countries']

## Model fitting

In [10]:
#Dates for filtering predictions
today = datetime.today()
last_day_week = today + timedelta(days=5 - today.weekday())
fist_d_next_week = last_day_week + timedelta(days=1)
last_d_next_week = last_day_week + timedelta(days=7)
first_next_month = datetime(year=(today.year + int(today.month % 12 == 0)), month=(today.month + 1) % 12, day=1)
last_next_month = datetime(year=(today.year + int(today.month % 12 == 0)), month=(today.month + 2) % 12, day=1) - timedelta(days=1)

df_output_predictions = pd.DataFrame() #predictions output dataframe

#counting iterations 
i=1
total_it = int(len(interest_items)*len(countries))

#number of days to keep on serie to plot. Ex: 60 will plot the previous 60 days + days_ahead below
n_days = 60
plot_start_date = today - timedelta(days=n_days)


for country in countries:
    for item in interest_items:
        if(country == "All countries" and item == "All items"):
            filter_in = np.ones(df.shape[0], dtype=bool)
        elif(country == "All countries"):
            filter_in = df["group1"] == item
        elif(item == "All items"):
            filter_in = df["country"] == country      
        else:
            filter_in = (df["country"] == country) & (df["group1"] == item)
        
        #creating temporary time series
        df_temp = get_time_series(df, filter_in = filter_in, frequency = "Daily")
        df_temp.bfill(axis ='rows')
        df_temp = df_temp.reset_index(drop=False)
        df_temp.columns = ["ds", "y"]
        
        
        #instantiating and fitting prophet
        model = Prophet()
        model.fit(df_temp)
        
        #Forecasting
        future = model.make_future_dataframe(periods=70, include_history=True)
        forecast = model.predict(future)
        
        #merging to get observed values
        forecast = pd.concat([forecast, df_temp["y"]], axis=1)
        
        #keeping only the columns I need
        forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', "y"]]
        
        #calculate statistics for next month/week here before dropping 
        
    
        #dropping forecast values to keep plot data
        #forecast = forecast[(forecast["ds"] >= plot_start_date) & (forecast["ds"] <= last_day_week)]
        
        n_rows = forecast.shape[0]
        forecast["item"] = [item]*n_rows
        forecast["country"] = [country]*n_rows
        
        df_output_predictions = df_output_predictions.append(forecast, ignore_index=True)
        print(f"Iteration {i} of {total_it}.") 
        i += 1
        

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Iteration 1 of 121.


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Iteration 2 of 121.


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Iteration 118 of 121.


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Iteration 119 of 121.
Iteration 120 of 121.


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Iteration 121 of 121.


In [11]:
df_output_predictions

,ds,yhat,yhat_lower,yhat_upper,y,item,country
0,2019-06-01,30.444337,8.714075,54.678137,3.0,Running shoes,Czech Republic
1,2019-06-03,50.872299,27.053598,74.911065,2.0,Running shoes,Czech Republic
2,2019-06-04,44.570893,19.316199,69.956806,71.0,Running shoes,Czech Republic
3,2019-06-05,40.891288,17.630358,65.270250,69.0,Running shoes,Czech Republic
4,2019-06-06,34.193648,8.376522,57.972754,21.0,Running shoes,Czech Republic
...,...,...,...,...,...,...,...
100038,2022-03-20,3296.365800,2796.240216,3771.739912,NaN,All items,All countries
100039,2022-03-21,3409.302309,2864.308133,3938.755923,NaN,All items,All countries
100040,2022-03-22,3255.433897,2756.795760,3754.193221,NaN,All items,All countries
100041,2022-03-23,3201.970110,2709.324900,3723.965607,NaN,All items,All countries


In [12]:
df_output_predictions.to_csv("df_output_predictions.csv")